# 9 top-notch Pandas Python functions for working with data

[Article link](https://nuancesprog.ru/p/14300/)

the dataframe was taken from ./data/sns-beginner/insurance.csv

In [4]:
# preparation
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('./data/sns-beginner/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## 1. Sort data in descending and ascending order

with func `sort_values()` we can sort not only one column, but also several, each in a different order (for example, one in ascending order, and the other in descending order)

In [18]:
df.sort_values(by=['age', 'bmi'], ascending=[True, False])

,age,sex,bmi,children,smoker,region,charges
1317,18,male,53.13,0,no,southeast,1163.46270
442,18,male,43.01,0,no,southeast,1149.39590
803,18,female,42.24,0,yes,southeast,38792.68560
781,18,male,41.14,0,no,southeast,1146.79660
728,18,female,40.28,0,no,northeast,2217.60120
...,...,...,...,...,...,...,...
1051,64,male,26.41,0,no,northeast,14394.55790
398,64,male,25.60,2,no,southwest,14988.43200
62,64,male,24.70,1,no,northwest,30166.61817
1265,64,male,23.76,0,yes,southeast,26926.51440


the column `'age'` was sorted with ascending, while the column `'bmi'` was sorted with descending 

In the same way you can create more sorting levels (for example three)

In [20]:
df.sort_values(by=['age', 'children', 'bmi'], ascending=[True, False, False])

,age,sex,bmi,children,smoker,region,charges
1095,18,female,31.350,4,no,northeast,4561.18850
369,18,male,30.400,3,no,northeast,3481.86800
1204,18,female,27.280,3,yes,southeast,18223.45120
46,18,female,38.665,2,no,northeast,3393.35635
564,18,female,32.120,2,no,southeast,2801.25880
...,...,...,...,...,...,...,...
402,64,female,32.965,0,no,northwest,14692.66935
890,64,female,26.885,0,yes,northwest,29330.98315
1051,64,male,26.410,0,no,northeast,14394.55790
1265,64,male,23.760,0,yes,southeast,26926.51440


## 2. `shift()` to move data

Let's say a situation requires shifting all rows in the dataframe or displaying the stock price of the previous day in it. We may be faced with the task of deriving the average temperature of the last three days. **Shift () is useful for working with time series data.**

> `DataFrame.shift(periods=1, freq=None, axis=0, fill_value=NoDefault.no_default)`

**Some parameters:**

**1. periods : int**

Number of periods to shift. Can be positive or negative.

**2. axis : {0 or ‘index’, 1 or ‘columns’, None}, default None**

Shift direction.

**3. fill_value : object, optional**

The scalar value to use for newly introduced missing values. the default depends on the dtype of self. For numeric data, `np.nan` is used. For datetime, timedelta, or period data, etc. `NaT` is used. For extension dtypes, `self.dtype.na_value` is used.

In [23]:
df = pd.DataFrame({'DATE': [1, 2, 3, 4, 5],
                   'VOLUME': [100, 200, 300,400,500],
                   'PRICE': [214, 234, 253,272,291]})

df

,DATE,VOLUME,PRICE
0,1,100,214
1,2,200,234
2,3,300,253
3,4,400,272
4,5,500,291


In [24]:
# simple shift()
df.shift(1)

,DATE,VOLUME,PRICE
0,NaN,NaN,NaN
1,1.0,100.0,214.0
2,2.0,200.0,234.0
3,3.0,300.0,253.0
4,4.0,400.0,272.0


![shift](./assets/images/shiftPD.png)

In [26]:
# with fill_value
df.shift(-1, fill_value=999)

,DATE,VOLUME,PRICE
0,2,200,234
1,3,300,253
2,4,400,272
3,5,500,291
4,999,999,999


If it is necessary to display the share price of the previous day in a new column

In [42]:
df['YESTERDAY_PRICE'] = df.PRICE.shift(-1, fill_value=0)
df

,DATE,VOLUME,PRICE,YESTERDAY_PRICE
0,1,100,214,234
1,2,200,234,253
2,3,300,253,272
3,4,400,272,291
4,5,500,291,0


We can easily calculate the average stock price for the last three days and create a new column as shown below:

In [43]:
df['LAST_3_DAYS_AVE_PRICE'] = (df['PRICE'].shift(1,fill_value=0) + 
                               df['PRICE'].shift(2,fill_value=0) + 
                               df['PRICE'].shift(3,fill_value=0))/3

df

,DATE,VOLUME,PRICE,YESTERDAY_PRICE,LAST_3_DAYS_AVE_PRICE
0,1,100,214,234,0.000000
1,2,200,234,253,71.333333
2,3,300,253,272,149.333333
3,4,400,272,291,233.666667
4,5,500,291,0,253.000000


Lets create TOMORROW_PRICE

In [46]:
df['TOMORROW_PRICE'] = df['PRICE'].shift(-1,fill_value=0)
df

,DATE,VOLUME,PRICE,YESTERDAY_PRICE,LAST_3_DAYS_AVE_PRICE,TOMORROW_PRICE
0,1,100,214,234,0.000000,234
1,2,200,234,253,71.333333,253
2,3,300,253,272,149.333333,272
3,4,400,272,291,233.666667,291
4,5,500,291,0,253.000000,0


## 3. Adding a new column at a given location in the dataframe

We use Pandas to create new columns for the dataframe quite often. By default, each such column is added to it from the end

> `DataFrame.insert(loc, column, value, allow_duplicates=False)`

Insert column into DataFrame at specified location.
Raises a ValueError if column is already contained in the DataFrame, unless allow_duplicates is set to True.

**Parameters:**

**1. loc : int**

Insertion index. Must verify 0 <= loc <= len(columns).

**2. column : str, number, or hashable object**

Label of the inserted column.

**3. value : int, Series, or array-like**

allow_duplicatesbool, optional

In [61]:
df = pd.DataFrame({'DATE': [1, 2, 3, 4, 5],
                   'VOLUME': [100, 200, 300,400,500],
                   'PRICE': [214, 234, 253,272,291]})

df['PRICE_PER_VOLUME'] = df.PRICE / df.VOLUME
df

,DATE,VOLUME,PRICE,PRICE_PER_VOLUME
0,1,100,214,2.140000
1,2,200,234,1.170000
2,3,300,253,0.843333
3,4,400,272,0.680000
4,5,500,291,0.582000


In [62]:
df = df.drop(columns='PRICE_PER_VOLUME')
df

,DATE,VOLUME,PRICE
0,1,100,214
1,2,200,234
2,3,300,253
3,4,400,272
4,5,500,291


If it is necessary **to create a column in a certain place of the dataframe**, for example, **between “VOLUME” and “PRICE”**, we will use the `insert` function.

In [63]:
df.insert(loc=2, column='PRICE_PER_VOLUME', value=(df.PRICE / df.VOLUME))
df

,DATE,VOLUME,PRICE_PER_VOLUME,PRICE
0,1,100,2.140000,214
1,2,200,1.170000,234
2,3,300,0.843333,253
3,4,400,0.680000,272
4,5,500,0.582000,291


## `value_counts()` for finding unique values

> `Series.value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)`

Return a Series containing counts of unique values.

**Parameters**

**1. normalize : bool, default False**

If True then the object returned will contain the relative frequencies of the unique values.

**2. sort : bool, default True**

Sort by frequencies.

**3. ascending : bool, default False**

Sort in ascending order.

**4. bins : int, optional**

Rather than count values, group them into half-open bins, a convenience for pd.cut, only works with numeric data.

**5. dropna : bool, default True**

Don’t include counts of NaN.

The Pandas `value_counts()` function returns an object containing the number of unique values. The resulting object can be sorted in descending or ascending order, including or excluding NA, through parameter control. This function is used with Pandas index or series.

![](./assets/images/value_counts_pd.png)

In [64]:
a = pd.Index([3,3,4,2,1,3, 1, 2, 3, 4, np.nan,4,6,7])
a.value_counts()

3.0    4
4.0    3
2.0    2
1.0    2
6.0    1
7.0    1
dtype: int64

In [65]:
b = pd.Series(['ab','bc','cd',1,'cd','cd','bc','ab','bc',1,2,3,2,3,np.nan,1,np.nan])
b.value_counts()


bc    3
cd    3
1     3
ab    2
2     2
3     2
dtype: int64

In [67]:
# with normalize
s = pd.Series([3, 1, 2, 3, 4, np.nan])
s.value_counts(normalize=True)

3.0    0.4
1.0    0.2
2.0    0.2
4.0    0.2
dtype: float64

In [69]:
# with intervals
s.value_counts(bins=3)

(0.996, 2.0]    2
(2.0, 3.0]      2
(3.0, 4.0]      1
dtype: int64

## 5. Selecting a column based on data type

In many cases, you need to select or perform certain operations based on the data type of the columns. Let's say our task is to apply a mask to all floating point integers, or convert all character columns to uppercase. There is one effective approach in Pandas for this purpose, the built-in **`select_dtypes`** function. It has options `include` and `exclude`, and in the form of a list, we can specify several options.

> `DataFrame.select_dtypes(include=None, exclude=None)`

Return a subset of the DataFrame’s columns based on the column dtypes.

In [72]:
df = pd.read_csv('./data/sns-beginner/insurance.csv')
df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [75]:
# include use
df.select_dtypes(include='object')

,sex,smoker,region
0,female,yes,southwest
1,male,no,southeast
2,male,no,southeast
3,male,no,northwest
4,male,no,northwest
...,...,...,...
1333,male,no,northwest
1334,female,no,northeast
1335,female,no,southeast
1336,female,no,southwest


In [78]:
# exclude use
df.select_dtypes(exclude=['object', 'int64']).dtypes

bmi        float64
charges    float64
dtype: object

In [81]:
# include all number types except float64 type
df.select_dtypes(include='number',exclude='float64')

,age,children
0,19,0
1,18,1
2,28,3
3,33,0
4,32,0
...,...,...
1333,50,3
1334,18,0
1335,18,0
1336,21,0


## 6. mask() for if-else conditions

The `mask()` method is the application of an `if-then condition` for each element of a series or dataframe. If `cond` is `True` then the value from other is used (the default is `NaN`), otherwise the original value is retained. **This method is similar to `where()`.**

![](./assets/images/mask_logic_pd.png)

> `DataFrame.mask(cond, other=nan, inplace=False, axis=None, level=None, errors='raise')`

Replace values where the condition is True.



**Parameters**

**1. cond : bool Series/DataFrame, array-like, or callable**

Where cond is False, keep the original value. Where True, replace with corresponding value from other. If cond is callable, it is computed on the Series/DataFrame and should return boolean Series/DataFrame or array. The callable must not change input Series/DataFrame (though pandas doesn’t check it).

**2. other : scalar, Series/DataFrame, or callable**

Entries where cond is True are replaced with corresponding value from other. If other is callable, it is computed on the Series/DataFrame and should return scalar or Series/DataFrame. The callable must not change input Series/DataFrame (though pandas doesn’t check it).

**3. inplace : bool, default False**

Whether to perform the operation in place on the data.

**4. axis : int, default None**

 Alignment axis if needed.

**5. level : int, default None**

Alignment level if needed.

**6. errors : str, {‘raise’, ‘ignore’}, default ‘raise’**

Note that currently this parameter won’t affect the results and will always coerce to a suitable dtype.

* ‘raise’ : allow exceptions to be raised.

* ‘ignore’ : suppress exceptions. On error return original object.

In [83]:
df = pd.DataFrame(np.arange(15).reshape(-1, 3), columns=['A', 'B','C'])
df

,A,B,C
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
4,12,13,14


you need to change the sign of all elements that are multiples of two without a remainder.

![](./assets/images/mask_use.png)

In [85]:
df.mask(df % 2 == 0,-df)

,A,B,C
0,0,1,-2
1,3,-4,5
2,-6,7,-8
3,9,-10,11
4,-12,13,-14


## 7. Filtering Columns Based on Partial Match

Let's say you want to display all columns containing “date” or “amount”. In such cases, the filter function is indispensable. In the dataframe under consideration, we will find all the columns that include “City”. In this case, you need to pay attention to the case of the matched strings, since it matters.

> `DataFrame.filter(items=None, like=None, regex=None, axis=None)[source]`

Subset the dataframe rows or columns according to the specified index labels.

Note that this routine does not filter a dataframe on its contents. The filter is applied to the labels of the index.

**Parameters**

**1. items : list-like**

Keep labels from axis which are in items.

**2. like : str**

Keep labels from axis for which “like in label == True”.

**3. regex : str (regular expression)**

Keep labels from axis for which re.search(regex, label) == True.

**4.axis : {0 or ‘index’, 1 or ‘columns’, None}, default None**

The axis to filter on, expressed either as an index (int) or axis name (str). By default this is the info axis, ‘index’ for Series, ‘columns’ for DataFrame.

![](./assets/images/filter_full.png)

![](./assets/images/filter_part.png)

In [89]:
df = pd.DataFrame(np.array(([1, 2, 3], [4, 5, 6])),
                  index=['mouse', 'rabbit'],
                  columns=['one', 'two', 'three'])

In [88]:
# select columns by regular expression
df.filter(regex='e$', axis=1)

,one,three
mouse,1,3
rabbit,4,6


In [91]:
# select rows containing 'bbi' (raBBIt in example)
df.filter(like='bbi', axis=0)

,one,two,three
rabbit,4,5,6


## 8. nlargest () to find the largest values

> `DataFrame.nlargest(n, columns, keep='first')`

Return the first n rows ordered by columns in descending order.

Return the first n rows with the largest values in columns, in descending order. The columns that are not specified are returned as well, but not used for ordering.

**This method is equivalent to df.sort_values(columns, ascending=False).head(n), but more performant.**

**Parameters**

**1. n : int**

Number of rows to return.

**2. columns : label or list of labels**

Column label(s) to order by.

**3. keep : {‘first’, ‘last’, ‘all’}, default ‘first’**

Where there are duplicate values:

* first : prioritize the first occurrence(s)

* last : prioritize the last occurrence(s)

* all : do not drop any duplicates, even it means

selecting more than n items.

It is often necessary to find the three highest or five lowest values ​​in a series or dataframe (for example, the top three students with their total score, or the three worst candidates with the total number of votes received in an election). Pandas provides `nlargest ()` and `nsmallest ()` just for this purpose.


The following is an example showing the 3 highest heights in a dataframe out of 10 available measurements:

![](./assets/images/nlargest_logic.png)

In [94]:
df = pd.DataFrame({'HEIGHT': [170,78,99,160,160,130,155,70,70,20],
                   'WEIGHT': [50,60,70,80,90,90,90,50,60,70]},
                   index=['A','B','C','D','E','F','G','H','I','J'])
df

,HEIGHT,WEIGHT
A,170,50
B,78,60
C,99,70
D,160,80
E,160,90
F,130,90
G,155,90
H,70,50
I,70,60
J,20,70


In [95]:
dfl = df.nlargest(3,'HEIGHT')
dfl

,HEIGHT,WEIGHT
A,170,50
D,160,80
E,160,90


If there are duplicate values, the options **first, last, all** help you choose the ones you want (default is **first**). Let's leave all three obtained options and try to find the 2 largest height values, as shown in the examples:

![](./assets/images/nlargest_options.png)

In [100]:
dfl = df.nlargest(2,'HEIGHT',keep='all')
dfl

,HEIGHT,WEIGHT
A,170,50
D,160,80
E,160,90


In [101]:
dfl = df.nlargest(2,'HEIGHT',keep='last')
dfl

,HEIGHT,WEIGHT
A,170,50
E,160,90


In [103]:
dfl = df.nlargest(2,'HEIGHT',keep='first')
dfl

,HEIGHT,WEIGHT
A,170,50
D,160,80


## 9. nsmallest()


nsmallest () works in a similar way, but only for the smallest values. Find the 2 smallest weights in the following examples:

In [106]:
df = pd.DataFrame({'HEIGHT': [170,78,99,160,160,130,155,70,70,20],
                   'WEIGHT': [50,60,70,80,90,90,90,50,60,70]},
                   index=['A','B','C','D','E','F','G','H','I','J'])
df

,HEIGHT,WEIGHT
A,170,50
B,78,60
C,99,70
D,160,80
E,160,90
F,130,90
G,155,90
H,70,50
I,70,60
J,20,70


In [107]:
dfs = df.nsmallest(3,'WEIGHT')
dfs

,HEIGHT,WEIGHT
A,170,50
H,70,50
B,78,60
